# Train a CNN model on sliding windows!

0. Data: MAPS
1. Pre-processing (on the fly!)
    1. Load the audio file
    2. 
    

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scipy import signal
import IPython.display as ipydisplay
import functools
import librosa
import librosa.display as ldisplay
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import scipy.io.wavfile as wav
%matplotlib inline

In [ ]:
from torch.autograd import Variable

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

### Dataset/dataloader

In [ ]:
def stft_features(signal):
    stft = librosa.stft(signal, n_fft=2048)
    stft_db = np.abs(stft)
    return stft_db

In [ ]:
dataset = SignalWindowDataset(
    folder_path='/home/anuj/data/ftps.tsi.telecom-paristech.fr/share/maps/ENSTDkCl/MUS/',
    feature_extractor=stft_features
)

In [ ]:
dataloader = DataLoader(dataset, batch_size=1)

In [ ]:
next(iter(dataloader))

stft = windows.transpose((1, 0, 2)).reshape(1025, -1)
stft_db = librosa.amplitude_to_db(np.abs(stft))
ldisplay.specshow(stft_db, x_axis='time')

### Bincounts

In [ ]:
from src import bincounts

In [ ]:
%%time
weights = bincounts.get_bin_counts(dataloader, keys=['labels'], n_iters=10)

In [ ]:
weights['labels']

In [ ]:
batch = next(iter(dataloader))
batch['windows'].transpose(0, 1).shape

In [ ]:
def predict_and_evalaute(batch, model, loss_func, device):
    inputs = batch['windows'].transpose(0, 1).to(device)
    target_labels = batch['labels'].to(device)

    # Predict
    label_probs = model(inputs)
    assert np.all(label_probs.shape == np.array([inputs.shape[0], 2]))

    pred_labels = torch.argmax(label_probs, dim=1)
    
    # loss
    loss = loss_func(label_probs, target_labels)

    return loss, pred_labels

### Model

In [ ]:
class SimpleCNN(torch.nn.Module):
    def __init__(self, in_dim=(1, 1025, 3), n_out=2,) -> None:
        super().__init__()
        nc, n_feats, n_t = in_dim
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(30, n_out)
        )

    def forward(self, windows):
        bsz, n_c, n_feats, n_t = windows.shape
        pass

In [ ]:
model = SimpleCNN()

### Loss / optimizer

In [ ]:
DEVICE = 'cuda:1'

In [ ]:
weights_l = Variable(torch.from_numpy(weights['labels'])).to(DEVICE)
loss_func = torch.nn.NLLLoss(weight=weights_l, ignore_index=-100)

In [ ]:
optim = torch.optim.Adam(params=model.parameters())

### Train